
<p align="center">
  <img src="https://storage.googleapis.com/kaggle-datasets-images/2243/3791/9384af51de8baa77f6320901f53bd26b/dataset-cover.png" />
  Image source: https://www.kaggle.com/
</p>

## Stage 1: Installing dependencies and setting up GPU environment

In [ ]:
!pip uninstall -y tensorflow

In [ ]:
!pip uninstall -y tensorflow-gpu

Uninstalling tensorflow-gpu-2.0.0a0:
  Successfully uninstalled tensorflow-gpu-2.0.0a0


In [ ]:
!pip install tensorflow-gpu==2.0.0.alpha0

  Using cached https://files.pythonhosted.org/packages/1a/66/32cffad095253219d53f6b6c2a436637bbe45ac4e7be0244557210dc3918/tensorflow_gpu-2.0.0a0-cp36-cp36m-manylinux1_x86_64.whl


In [ ]:
!pip show tensorflow-gpu

Name: tensorflow-gpu
Version: 2.0.0a0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: six, grpcio, google-pasta, numpy, protobuf, tb-nightly, gast, termcolor, keras-preprocessing, tf-estimator-nightly, wheel, astor, keras-applications, absl-py
Required-by: 


## Stage 2: Import dependencies for the project

In [ ]:
import numpy as np
import datetime
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

In [ ]:
tf.__version__

'2.2.0-rc3'

## Stage 3: Dataset preprocessing



### Loading the dataset

In [ ]:
#Loading the Fashion Mnist dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


### Image normalization

We devide each image in the training and testing dataset with the maxiumum number of pixels (255).

In this way each pixel will be in the rainge [0, 1]. By normalizing imaes we are making sure that our model (ANN) trains faster.

In [ ]:
X_train = X_train / 255.0

In [ ]:
X_test = X_test / 255.0

### Reshaping of the dataset

Since we are using fully connected network, we reshape the training and testing subsets to be in the vector format. 

In [ ]:
#Since each image is 28x28, we simply use reshape the full dataset to [-1 (all elements), height * width]
X_train = X_train.reshape(-1, 28*28)

In [ ]:
X_train.shape

(60000, 784)

In [ ]:
#Reshape the testing subset in the same way
X_test = X_test.reshape(-1, 28*28)

## Stage 4: Building an Artificial Neural network

### Defining the model

Simply define an object of the Sequential model.

In [ ]:
model = tf.keras.models.Sequential()

### Adding the first layer (Dense layer)

Layer hyper-parameters:
- number of units/neurons: 128
- activation function: ReLU
- input_shape: (784, )

In [ ]:
model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784, )))

### Adding a Dropout layer 

Dropout is a Regularization technique where we randomly set neurons in a layer to zero. In this way, while training those neurons won't be updated. Because some percentage of neurons won't be updated the whole training process is long and we have less chance for overfitting.

In [ ]:
model.add(tf.keras.layers.Dropout(0.2))

### Adding the second layer (output layer)

- units == number of classes (10 in the case of Fashion MNIST)
- activation = 'softmax'

In [ ]:
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

### Comiling the model

- Optimizer: Adam
- Loss: Sparse softmax (categorical) crossentropy 

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


### Training the model

In [ ]:
%%time
model.fit(X_train, y_train, epochs=5, batch_size = 128)

Epoch 1/5
469/469 [==============================] - 2s 3ms/step - loss: 0.2908 - sparse_categorical_accuracy: 0.8941
Epoch 2/5
469/469 [==============================] - 1s 3ms/step - loss: 0.2860 - sparse_categorical_accuracy: 0.8949
Epoch 3/5
469/469 [==============================] - 1s 3ms/step - loss: 0.2833 - sparse_categorical_accuracy: 0.8961
Epoch 4/5
469/469 [==============================] - 2s 3ms/step - loss: 0.2814 - sparse_categorical_accuracy: 0.8962
Epoch 5/5
469/469 [==============================] - 1s 3ms/step - loss: 0.2739 - sparse_categorical_accuracy: 0.8989
CPU times: user 8.49 s, sys: 997 ms, total: 9.49 s
Wall time: 7.81 s


### Model evaluation and prediction

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3252 - sparse_categorical_accuracy: 0.8858


In [ ]:
print("Test accuracy: {}".format(test_accuracy))

Test accuracy: 0.8858000040054321


## Stage 5 : Saving the model

### Saving the architecture (topology) of the network

In [ ]:
model_json = model.to_json()
with open("fashion_model.json", "w") as json_file:
    json_file.write(model_json)

### Saving network weights

In [ ]:
model.save_weights("fashion_model.h5")